# Modeling - Given these features, what model works best?

## Here you can actually test assumptions
1. Linearity by residuals vs predicted plot.
2. Homoscedasticity with spread of residuals vs predicted.
3. Normality of errors with Q-Q plot of residuals.
4. Independence with residuals vs time/order.
5. Multicollinearity with VIF, condition number.
6. Influential points with Cook's distance.

## They require predictions, residuals, fitted model parameters so they belong to modeling.ipynb.

In [1]:
# 3rd Party
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# linear_regression_001
from linear_regression_001.data.loader import load_raw_data
from linear_regression_001.features.build_features import split_features_target, build_features, build_preprocessor, FEATURE_LIST
from linear_regression_001.utils.paths import INTERIM, PROCESSED, INFERENCE, PREDICTIONS, MODELS
from linear_regression_001.models import train_models, predict_features, evaluate_model

# Baseline Model
from sklearn.dummy import DummyRegressor

# Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import r2_score as r2

In [2]:
df = load_raw_data("insurance.csv")
X, y = split_features_target(df)
X

*normalized*
*missing checked*
*schema enforced*


,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest
1334,18,female,31.920,0,no,northeast
1335,18,female,36.850,0,no,southeast
1336,21,female,25.800,0,no,southwest


In [3]:
evaluate_model(MODELS / "linear_regression.pkl",INTERIM / "test.csv", MODELS / "baseline.pkl")

{'metrics': {'rmse': np.float64(4573.807686290385),
  'mae': 2756.898659517957,
  'r2': 0.8652503208873077,
  'mape': 29.24770582699016,
  'adjusted_r2': 0.8616224449111968},
 'baseline_comparison': {'baseline_rmse': np.float64(12465.610441715768),
  'model_rmse': np.float64(4573.807686290385),
  'improvement_percent': np.float64(-63.30859441119478),
  'beats_baseline': np.True_}}

In [3]:
X_features_built = build_features(X,FEATURE_LIST)
X_features_built

,age,bmi,children,region,smoker,sex,bmi_smoker
0,19,27.900,0,southwest,yes,female,27.90
1,18,33.770,1,southeast,no,male,0.00
2,28,33.000,3,southeast,no,male,0.00
3,33,22.705,0,northwest,no,male,0.00
4,32,28.880,0,northwest,no,male,0.00
...,...,...,...,...,...,...,...
1333,50,30.970,3,northwest,no,male,0.00
1334,18,31.920,0,northeast,no,female,0.00
1335,18,36.850,0,southeast,no,female,0.00
1336,21,25.800,0,southwest,no,female,0.00


In [4]:
# build dataset and preprocessor
df = pd.concat([X_features_built,y],axis=1)
preprocessor = build_preprocessor()
df

,age,bmi,children,region,smoker,sex,bmi_smoker,charges
0,19,27.900,0,southwest,yes,female,27.90,16884.92400
1,18,33.770,1,southeast,no,male,0.00,1725.55230
2,28,33.000,3,southeast,no,male,0.00,4449.46200
3,33,22.705,0,northwest,no,male,0.00,21984.47061
4,32,28.880,0,northwest,no,male,0.00,3866.85520
...,...,...,...,...,...,...,...,...
1333,50,30.970,3,northwest,no,male,0.00,10600.54830
1334,18,31.920,0,northeast,no,female,0.00,2205.98080
1335,18,36.850,0,southeast,no,female,0.00,1629.83350
1336,21,25.800,0,southwest,no,female,0.00,2007.94500


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features_built,y,test_size=0.2,random_state=42)

In [6]:
# Setup pipeline
model = Pipeline(steps=[
        ("preprocessing", preprocessor),
        ("regressor", LinearRegression())
    ])

# Train linear regression model
lr = model.fit(X_train, y_train)

# Train baseline dummy model
baseline = DummyRegressor(strategy = "mean")
baseline.fit(X_train,y_train)

,"strategy strategy: {""mean"", ""median"", ""quantile"", ""constant""}, default=""mean""Strategy to use to generate predictions.* ""mean"": always predicts the mean of the training set* ""median"": always predicts the median of the training set* ""quantile"": always predicts a specified quantile of the training set, provided with the quantile parameter.* ""constant"": always predicts a constant value that is provided by the user.",'mean'
,"constant constant: int or float or array-like of shape (n_outputs,), default=NoneThe explicit constant as predicted by the ""constant"" strategy. Thisparameter is useful only for the ""constant"" strategy.",None
,"quantile quantile: float in [0.0, 1.0], default=NoneThe quantile to predict using the ""quantile"" strategy. A quantile of0.5 corresponds to the median, while 0.0 to the minimum and 1.0 to themaximum.",None


In [ ]:
preds = predict_features("predict_data.csv",index=False)

Inference completed. Predictions saved.


In [9]:
preds

array([11248.03667869,  7416.9145174 , 33065.59607691, 10829.97561501,
       28195.54289901,  6487.37519279,  2787.29181519, 14426.29848503,
        5683.41343095, 11679.78604499, 20668.61540699,  9147.48495868,
        5692.06090832, 51914.29540262, 52078.51416297, 46629.33697172,
       11688.60874331, 43246.44342887, 10138.51523512, 27534.39305694,
        5586.70513748,  9112.96028181,  1819.72842257,  4559.01619765,
       12456.86456988, 12843.22917473, 13239.15339896,  6445.05210603,
       11803.67529126,  1817.0185446 ,  9708.49692225, 12984.6702093 ,
        3228.20748468,  7023.60323381,  4119.01507054, 10091.7696037 ,
        3254.38998848,  9119.86138565, 26260.54405807, 37691.6237046 ,
        4626.77097393,  3694.80115433, 12663.41909349, 12857.32433121,
        6808.79001016, 12760.27856106,  4466.46580745,  5192.73106033,
       41610.82054866,  6335.47084988, 14319.57215113,  2265.07806111,
        9164.98860108,  2703.19208702, 11912.73798409, 12180.19813797,
      

In [ ]:
# Train both baseline and LinearRegression -- compact
# train_models()

*normalized*
*missing checked*
*schema enforced*
Training complete. Models saved.


In [ ]:
# Predict with Model
# Open models
baseline_model = joblib.load(MODELS / "baseline.pkl")
linear_regression_model = joblib.load(MODELS / "linear_regression.pkl")


In [3]:
# Baseline Model -- Predict Mean of y or simple linear regression w/o tuning
baseline_model = DummyRegressor(strategy='mean') # predicts the mean of y_train
baseline_model.fit(X_train,y_train)
baseline_preds = baseline_model.predict(X_test)

In [4]:
# Baseline Evaluation
evaluate(y_test, baseline_preds, "Baseline Model")

Baseline Model Performance
-------------------------
MAE: 9861.7979
RMSE: 13612.4260
R^2: -0.0084


In [ ]:
# Train Model

In [ ]:
# Evaluation